In [ ]:
#imports

import sqlite3
import tkinter
from tkinter import * #to use every options in tkinter
from tkinter import filedialog 
from tkinter import messagebox 
import PIL 
from PIL import Image
from PIL import ImageTk, Image
from PIL import ImageFilter
from PIL.ImageFilter import (MedianFilter,SHARPEN)
#import tkinter as Tk



# make database and users (if not exists already) table at programme start up
with sqlite3.connect('quit.db') as db:
    c = db.cursor()

c.execute('CREATE TABLE IF NOT EXISTS user (username TEXT NOT NULL ,password TEX NOT NULL);')
db.commit()
db.close()

#main Class
class main:
    def __init__(self,master):
    	# Window 
        self.master = master
        # Some Usefull variables
        self.username = StringVar()
        self.password = StringVar()
        self.n_username = StringVar()
        self.n_password = StringVar()
        #Create Widgets
        self.widgets()

    #Login Function
    def login(self):
    	#Establish Connection
        with sqlite3.connect('quit.db') as db:
            c = db.cursor()

        #Find user If there is any take proper action
        find_user = ('SELECT * FROM user WHERE username = ? and password = ?')
        c.execute(find_user,[(self.username.get()),(self.password.get())])
        result = c.fetchall()
        if result:
            self.logf.pack_forget()
            window = Toplevel(logwindow)
            #self.head['text'] = self.username.get() + '\n Loged In'
            #self.head['pady'] = 150
    
            #app Class Object
            class application:

                def __init__(self,master):
                    self.master = master
                    self.c_size = (500,300)
                    self.setup_gui(self.c_size)
                    self.img=None

    
                def setup_gui(self,s):
                    #gui title
                    Label(self.master,text
                          ='                                     Leukemia detection and classification ',
                          pady=5,font="times 25",padx=20).grid(columnspan=4, row=0, sticky=W)
                    #set place of image frame 
                    self.canvas = Canvas(self.master,height=s[1],width=s[0],
                        bg='white',bd=10,relief='ridge')
                    self.canvas.grid(column=1,row=5,columnspan=3,rowspan=3)
                    txt = '''          
                                                             Image
                    '''
                    self.wt = self.canvas.create_text(s[0]/2-270,s[1]/2,text=txt
                        ,font=('',30),fill='white')

                    #creat buttons to import,improve,classify image        
                    menu=LabelFrame(window,text='Menu',borderwidth=1,font="25", width=300, height=130, bd=5)
                    menu.grid(row=4,column=0,rowspan=4,sticky=E+S+W+N)
                    Image_but=Button(window,text='Import Image',font=("15"), bg='gray' ,command=self.make_image).grid(column=0,row=5)
                    improve_button = Button(window, text="Improve Image",font="15 ",bg='gray',command=self.improved).grid(column=0,row=6)
                    classfy_button = Button(window, text="Classify Image",font="15 ",bg='gray',command= self.test).grid(column=0,row=7)


                    #set place of status lable to show the path of selected image
                    self.status=Label(self.master,text = 'Current Image: None',font='times 15',bd=2,fg='black',relief='solid')
                    self.status.grid(sticky=S,row=11,column=0,columnspan=4)
                
                

                    #bulid function to display image
                def make_image(self):
                    global File
                    try:
                     #set file dialog to user to choose image 
                        File = filedialog.askopenfilename(parent=window,
                            initialdir='D:\\leukemia\\data\\Data ',
                            title='Choose image',
                            filetypes=[("all images","*.*")]
                                                 )
                        self.pilImage = Image.open(File) # open image
                        resize=self.pilImage.resize((256,256),Image.ANTIALIAS) # resize image           
                        self.img = ImageTk.PhotoImage(resize)
                        self.canvas.delete(ALL)
                        self.canvas.create_image(self.c_size[0]/2+10,self.c_size[1]/2+10,
                            anchor=CENTER,image=self.img)  #creat image in convas 

                        self.status['text']='Current Image:'+File #show image file (path)
                    except:
                        messagebox.showerror('Error!','File type is unsupported.') #masegebox 


                    #built function of filters to improve image quality 
                def improved(self):
                    global File
                    global median_images
                    try:
                        self.pilImage =Image.open(File) #open the same selected image to improve it
                         #selected filters
                        #resized = self.pilImage.resize((128,128),Image.ANTIALIAS) 
                        sharp_images =self.pilImage .filter(SHARPEN)
                        median_images =sharp_images .filter(MedianFilter(size=1))

                        self.img = ImageTk.PhotoImage(median_images)
                        self.canvas.delete(ALL)
                        self.canvas.create_image(self.c_size[0]/2+10,self.c_size[1]/2+10,
                                anchor=CENTER,image=self.img) #creat image in convas 
                    except:
                         messagebox.showerror('Error!','PLEASE CHOOSE AN IMAGE') #masegebox

                     #built function to classify image

                def test(self):
                    from keras.models import load_model
                    global median_images
                    import numpy as np
                    global res_var

                    model = load_model('model.h5')

                    self.resized = median_images.resize((128,128),Image.ANTIALIAS)
                    self.test_image = np.array(self.resized).reshape(-1,128, 128,3)
                    self.test_image = self.test_image.astype('float32') 
                    self.test_image /= 255

                    self.result = model.predict_classes(self.test_image)

                    if self.result == 0 :
                        self.result='cancer'
                    else :
                        self.result='normal'


                    
                    #creat result normal/abnormal
                    res_var=StringVar()
                    res_label=Label(window,textvariable=res_var,font="times 20 bold",borderwidth=1,relief='solid')
                    res_label.grid(column=4, row=6)
                    res_var.set(self.result)#cheack that is aplace for variable assined to it
                    
                # Quit button in the lower right corner
                import tkinter 
                from tkinter import messagebox

                def ExitApplication():
                    MsgBox = messagebox.askquestion ('Exit Application','Are you sure you want to exit the application',icon = 'warning')
                    if MsgBox == 'yes':
                        window.destroy()
                    else:
                        messagebox.showinfo('Return','You will now return to the application screen')

                quit_button = Button(window, text="Quit",font="times 20 bold",borderwidth=1,bg='gray',width=8,command=ExitApplication)
                quit_button.grid(column=4,row=9)
                
                   #################################################################################################################
               
                #creat new window of report
                def create_window():
                    from datetime import date
                    from datetime import time
                    from datetime import datetime
                    
                    window2 = Toplevel(window)
                    window2.title("Report")
                    #window2.config(bg='white')
                    title=Label(window2,text='        Result report  ',
                          pady=5,font="times 40",padx=20).grid(column=1, row=0, sticky=W)
                    
                    #images lable
                    load = Image.open(r"C:\Users\NoteBook\Desktop\OurData\classes\uni.jpg")
                    render = ImageTk.PhotoImage(load)
                    img = Label(window2, image=render)
                    img.image = render
                    img.grid(column=0,row=0)
                    
                    load = Image.open(r"C:\Users\NoteBook\Desktop\OurData\classes\dep.jpg")
                    render = ImageTk.PhotoImage(load)
                    img = Label(window2, image=render)
                    img.image = render
                    img.grid(column=3,row=0)
                    
                    #date
                    Date=Label(window2,text='Date',font="times 20",padx=20,pady=20).grid(column=0, row=1)
                    today=IntVar()
                    today_lable=Label(window2,textvariable=today,font="times 20 bold",borderwidth=1,padx=20).grid(column=1, row=1)
                    today.set(date.today())
                    
                    #time
                    Time=Label(window2,text='Time',font="times 20",padx=20).grid(column=2, row=1)
                    timeN=IntVar()
                    timeN_lable=Label(window2,textvariable=timeN,font="times 20 bold",borderwidth=1,padx=20).grid(column=3, row=1)
                    now=datetime.now() 
                    timeN.set(now.strftime("%I:%M:%S %p"))
                    
                    #show name of patient behend lable
                    patient_name=Label(window2,text='Patient name',font="times 20",pady=20).grid(column=0, row=2)
                    global name_inp
                    name_var=StringVar()
                    name_label=Label(window2,textvariable=name_inp,font="times 20 bold",borderwidth=1,padx=20).grid(column=1, row=2)
                    name_var.set(name_inp)
                    
                    #show age of patient behend patient lable
                    patient_age=Label(window2,text='Patient age',font="times 20",padx=20,pady=20).grid(column=0, row=3)
                    global age_inp
                    age_var=IntVar()
                    age_label=Label(window2,textvariable=age_inp,font="times 20 bold").grid(column=1, row=3)
                    age_var.set(age_inp)

                    #show result of patient behend lable
                    patient_result=Label(window2,text='Test result',font="times 20",padx=20,pady=20).grid(column=0, row=4)
                    global res_var
                    rest_var=StringVar()
                    rest_label=Label(window2,textvariable=res_var,font="times 20 bold",padx=20).grid(column=1, row=4)
                    rest_var.set(res_var)

                     #################################################################################################################  
                #creat report button
                global name_inp
                global age_inp
                report_button = Button(window, text="Report",font="times 20 bold",borderwidth=1,bg='gray',width='8',command=create_window)
                report_button.grid(column=4,row=7)

                
                #name ,age,gender
                import tkinter as tk
                patient_inf=LabelFrame(window,text='Patient Information',font="times 15" , width=80, height=180, bd=5)
                patient_inf.grid(row=1,column=0,columnspan=5,rowspan=3,sticky=E+W+N+S)

                global name_inp
                name=Label(window,text='Name',borderwidth=1,font="15")
                name.grid(row=2,column=0)

                name_inp = StringVar()
                name_entry = Entry(window ,textvariable=name_inp,bd =3 ,font = ('',15))
                name_entry.grid(row=2,column=1)

                global age_inp
                age=Label(window,text='Age',borderwidth=1,font="15")
                age.grid(row=2,column=2)

                age_inp = IntVar()
                age_entry = Entry(window ,textvariable=age_inp,bd =3 ,font = ('',15))
                age_entry.grid(row=2,column=3)

                gender=Label(window,text='Gender',borderwidth=1,font="15")
                gender.grid(row=3,column=0)

                v = tk.IntVar()
                male=tk.Radiobutton(window,text="Male", variable=v,value=1,font="times 15").grid(row=3,column=1)
                female=tk.Radiobutton(window, text="Female", variable=v, value=2,font="times 15").grid(row=3,column=3)

    
            #creat output parameters labels
            output=LabelFrame(window,text='Output Parameters',font="25 ",borderwidth=1,padx=20,width=300, height=130, bd=5)
            output.grid(row=4,column=4,sticky=E+W+N+S,rowspan=3)

            result=Label(window,text='Result',font="times 20",width=10,borderwidth=1)
            result.grid(row=5, column=4)

            #creating object of class and tk window-
            window.geometry=(1200,600)
            window.title('leukemia detection and classification') #window title
            application(window)

            window.columnconfigure(0, weight=1)
            window.rowconfigure(0, weight=1)

        else:
            messagebox.showerror('Oops!','Username Not Found.')
            
    def new_user(self):
    	#Establish Connection
        with sqlite3.connect('quit.db') as db:
            c = db.cursor()

        #Find Existing username if any take proper action
        find_user = ('SELECT * FROM user WHERE username = ?')
        c.execute(find_user,[(self.username.get())])        
        if c.fetchall():
            messagebox.showerror('Error!','Username Taken Try a Diffrent One.')
        else:
            messagebox.showinfo('Success!','Account Created!')
            self.log()
        #Create New Account 
        insert = 'INSERT INTO user(username,password) VALUES(?,?)'
        c.execute(insert,[(self.n_username.get()),(self.n_password.get())])
        db.commit()

        #Frame Packing Methords
    def log(self):
        self.username.set('')
        self.password.set('')
        self.crf.pack_forget()
        self.head['text'] = 'LOGIN'
        self.logf.pack()
    def cr(self):
        self.n_username.set('')
        self.n_password.set('')
        self.logf.pack_forget()
        self.head['text'] = 'Create Account'
        self.crf.pack()
        
    #Draw Widgets
    def widgets(self):
        self.head = Label(self.master,text = 'LOGIN',font = ('',35),pady = 10)
        self.head.pack()
        self.logf = Frame(self.master,padx =10,pady = 10)
        Label(self.logf,text = 'Username: ',font = ('',20),pady=5,padx=5).grid(sticky = W)
        Entry(self.logf,textvariable = self.username,bd = 5,font = ('',15)).grid(row=0,column=1)
        Label(self.logf,text = 'Password: ',font = ('',20),pady=5,padx=5).grid(sticky = W)
        Entry(self.logf,textvariable = self.password,bd = 5,font = ('',15),show = '*').grid(row=1,column=1)
        Button(self.logf,text = ' Login ',bd = 3 ,font = ('',15),padx=5,pady=5,command=self.login,bg='gray').grid()
        Button(self.logf,text = ' Create Account ',bd = 3 ,font = ('',15),padx=5,pady=5,command=self.cr,bg='gray').grid(row=2,column=1)
        self.logf.pack()
        
        self.crf = Frame(self.master,padx =10,pady = 10)
        Label(self.crf,text = 'Username: ',font = ('',20),pady=5,padx=5).grid(sticky = W)
        Entry(self.crf,textvariable = self.n_username,bd = 5,font = ('',15)).grid(row=0,column=1)
        Label(self.crf,text = 'Password: ',font = ('',20),pady=5,padx=5).grid(sticky = W)
        Entry(self.crf,textvariable = self.n_password,bd = 5,font = ('',15),show = '*').grid(row=1,column=1)
        Button(self.crf,text = 'Create Account',bd = 3 ,font = ('',15),padx=5,pady=5,command=self.new_user,bg='gray').grid()
        Button(self.crf,text = 'Go to Login',bd = 3 ,font = ('',15),padx=5,pady=5,command=self.log,bg='gray').grid(row=2,column=1)
   
    

#create window and application object
logwindow = Tk()
logwindow.title("Login Form")
main(logwindow)
logwindow.mainloop()
